## Using Komo for IK

In [1]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

**ry-c++-log** /home/jung-su/git/robotics-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



In [2]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

In [3]:
obj = C.addFrame("object")
obj.setPosition([.8,0,1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

In [4]:
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
obj.setContact(1)

In [5]:
IK = C.komo_IK(False)
IK.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
IK.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2]);

In [6]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()

{'F_qItself/1-#28': {'order': 1.0, 'type': 'sos', 'sos': 0.004653745738764228},
 'F_qQuaternionNorms/0-#84': {'order': 0.0, 'type': 'eq'},
 'F_AccumulatedCollisions/0-#84': {'order': 0.0, 'type': 'ineq'},
 'F_PositionDiff/0-R_gripperCenter-object': {'order': 0.0,
  'type': 'sos',
  'sos': 7.457660878364029e-07},
 'sos': 0.004654491504852064,
 'ineq': 0.0,
 'eq': 0.0,
 'f': 0.0}

In [7]:
C.setFrameState( IK.getFrameState(0) )

In [8]:
# Move object and reoptimize

# move object
obj.setPosition([.2,.2,1.5])

# # copy C into the IK's internal configuration(s)
# IK.setConfigurations(C)

# redefine the IK problem
IK = C.komo_IK(False)
IK.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
IK.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2]);

# reoptimize
IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getFrameState(0) )

{'F_qItself/1-#28': {'order': 1.0, 'type': 'sos', 'sos': 0.00614054649680664}, 'F_qQuaternionNorms/0-#84': {'order': 0.0, 'type': 'eq'}, 'F_AccumulatedCollisions/0-#84': {'order': 0.0, 'type': 'ineq'}, 'F_PositionDiff/0-R_gripperCenter-object': {'order': 0.0, 'type': 'sos', 'sos': 4.176652243530601e-08}, 'sos': 0.006140588263329075, 'ineq': 0.0, 'eq': 0.0, 'f': 0.0}


## Path Optimization

In [9]:
obj.setPosition([.8,.2,1.5])

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,20, 5., True)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2])
komo.optimize()
komo.getReport()

{'F_qItself/2-#28': {'order': 2.0, 'type': 'sos', 'sos': 0.06415076629917972},
 'F_AccumulatedCollisions/0-#84': {'order': 0.0,
  'type': 'ineq',
  'ineq': 0.6916534719577405},
 'F_PositionDiff/0-R_gripperCenter-object': {'order': 0.0,
  'type': 'sos',
  'sos': 0.0004416415850860006},
 'sos': 0.06459240788426572,
 'ineq': 0.6916534719577405,
 'eq': 0.0,
 'f': 0.0}

In [10]:
V = komo.view_play(True, 0.1)

In [11]:
C.setFrameState(komo.getFrameState(19))
C.getJointState()
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)

(array([0.]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]))